In [1]:
"""
This module contains code for scraping data from the Rexel website and processing the scraped data.
"""

import json
from time import sleep

import pandas as pd
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service

with open('config.json', 'r', encoding='utf-8') as f:
    config = json.load(f)

options = Options()
options.add_argument("--disable-logging")
options.add_argument("--start-maximized")

service = Service()
driver = webdriver.Chrome(service=service, options=options)
driver.get('https://rexel.it/login')

# Find the username and password input fields and enter your credentials
username_field = driver.find_element('id', 'MainContent_wucLoginBoxes_UserName')
username_field.send_keys(config['username'])

password_field = driver.find_element('id', 'MainContent_wucLoginBoxes_Password')
password_field.send_keys(config['password'])

# Submit the login form
login_button = driver.find_element('id', 'lbLogin')
login_button.click()

driver.implicitly_wait(15)

NO_OF_PAGES = 884
COUNT = 1
products_list = []

In [3]:
while COUNT < NO_OF_PAGES:
    driver.get(f'https://rexel.it/categoria/illuminazione?page={COUNT}')

    # Wait for the page to load
    sleep(15)

    page_source = driver.page_source
    soup = BeautifulSoup(page_source, 'html.parser')
    products = soup.select('.col-product')

    for product in products:
        name = product.select_one(".name-product").text.strip()
        code = product.select_one(".code").text.strip()
        OFFER = product.select_one(".alert")

        if OFFER is not None:
            OFFER = OFFER.text.strip()
            if 'Offerta limitata' in OFFER and 'A solo: € 0,00' not in OFFER:
                price = OFFER.split('\n')[2].replace('A solo: ','').strip()
                OFFER = 'Offerta limitata'
                # Create a dictionary for the product
                product_dict = {
                    "Product Name": name,
                    "Price": price.split('x')[0].rstrip(),
                    "Code": code.split('Cod. Produttore')[0].strip(),
                }
                products_list.append(product_dict)
                print(f"Product Name: {name}")
                print(f"Price: {price.split('x')[0].rstrip()}")
                print(f"{code.split('Cod. Produttore')[0].strip()}")
                print("---")
            else: OFFER ='NA'

    if COUNT%5 == 0:
        print(f'SUCCESSFULLY SCRAPED PAGE {COUNT}')
    COUNT+=1

with open("products.json", "w", encoding='utf-8') as json_file:
    json.dump(products_list, json_file)

driver.quit()

In [5]:
df = pd.DataFrame.from_dict(products_list)
df.to_excel("Prodotti in Offerta.xlsx", index= False)

ModuleNotFoundError: No module named 'pandas'